# 1.1 shopbop crawling:
creating functiones to take: rating, size, fabric, color, name, pricre and id.

runing through all links of the website and creating a DF with the above, brand name and numerical order.

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time 
import urllib.request
import requests
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

from selenium.webdriver.common.by import By

In [2]:
driverS = webdriver.Chrome(ChromeDriverManager().install())
driverS.get('https://www.shopbop.com/clothing/br/v=1/13266.htm') 

[WDM] - Downloading: 100%|██████████| 6.58M/6.58M [00:01<00:00, 6.86MB/s]
C:\Users\linoy\AppData\Local\Temp\ipykernel_21912\3423180645.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driverS = webdriver.Chrome(ChromeDriverManager().install())


In [9]:
def ratingS(soup):
    star=soup.find("meta", {"itemprop":"ratingValue"})
    if(star!= None):
        rating= star.attrs['content']
    else:
        rating=0
    return rating


def sizeS(soup):
    div=soup.find("div", {"id":"sizeList"})
    size_tags=div.find_all("div", {"class":"sizeBox"})
    if size_tags==None: return None
    size=[]
    for s in size_tags:
        size.append(s.string)
    return size

def fabricS(soup):
    div=soup.find("div", {"class":"Details__Section-sc-3us40t-0 bXftpn"})
    if div==None: return None
    fabric=div.find("span")
    if fabric==None: return None
    return fabric.string

def colorS(soup):
    div=soup.find("div", {"class":"colors"})
    color=div.find_all("img")
    if color==None: return None
    colors_list=[]
    for c in color:
        colors_list.append(c.attrs['alt'])
    return colors_list

def name_price_id(soup):
    n=p=i=None
    
    name=soup.find("div", {"id":"product-title"})
    if name!=None: 
        n=name.string
    
    price=soup.find("meta", {"itemprop":"lowPrice"})
    if price!=None: 
        p=price.attrs['content']
    
    idp=soup.find("span", {"itemprop":"sku"})
    if idp!=None: 
        i=idp.string
        
    return n, p, i


In [1]:
dfS=pd.read_csv("shopbop_link_test3.csv")

ratings=[]
prices=[]
product_names=[] 
brands=[] 
colors=[] 
sizes=[] 
fabrics=[] 
ids=[]

categorys=[] 

row = dict()
temp_list=[]

for num, s in enumerate(dfS["link S"]):
    driverS.get(s) 
    time.sleep(4)
    
    row = dict()
    row['num']=num
    soup = BeautifulSoup(driverS.page_source, 'html.parser')
    brands.append("shopbop")
    row['brand']= "shopbop"
    
    name, price, idp= name_price_id(soup)

    product_names.append(name)
    row['product_names']= name

    prices.append(price)
    row['prices']= price

    ids.append(idp)
    row['ids']= idp
    
    rating=ratingS(soup)
    ratings.append(rating)
    row['ratings']= rating
    
    size=sizeS(soup)
    sizes.append(size)
    row['sizes']= size
    
    fabric=fabricS(soup)
    fabrics.append(fabric)
    row['fabrics']= fabric
    
    color=colorS(soup)
    colors.append(color)
    row['colors']= color
    
    temp_list.append(row)
    
    if num%100==0:
        df = pd.DataFrame(temp_list)
        df.to_csv("shopbop_prodoct_info_temp2.csv", index=False)


In [14]:
df

,num,brand,product_names,prices,ids,ratings,sizes,fabrics,colors
0,0,shopbop,Lace Top,490.00,PACOO30471,0,"[\n 34, \n ...","Fabric: Lightweight, stretch lace",[Black]
1,1,shopbop,Jupe Skirt,620.00,PACOO30468,0,"[\n 34, \n ...","Fabric: Lightweight, stretch floral lace",[Black]
2,2,shopbop,Ella Dress,285.00,XIREN30738,0,"[\n XS, \n ...",None,"[Electric Blue, Rococo Rose]"
3,3,shopbop,Rosalie Dress,840.00,LGVSG30040,0,"[\n 34, \n ...",None,[Black]
4,4,shopbop,Wyatt Shirt,255.00,RAGBO46380,0,"[\n XXS, \n ...","Fabric: Mid-weight, non-stretch denim",[Vintage Blue]
...,...,...,...,...,...,...,...,...,...
7596,7596,shopbop,Orchid Lace Slip,175.00,FDUMA30699,0,"[\n XS, \n ...","Fabric: Mid-weight, body-con jersey knit",[Black]
7597,7597,shopbop,Rio Blazer Mini Dress,435.00,BLACK30972,5.0,"[\n 0, \n ...",Fabric: Mid-weight stretch suiting,[Mermaid's Kiss]
7598,7598,shopbop,Rib Cyrene Pants,170.00,SIMIL30384,0,"[\n XXS, \n ...",Fabric: Mid-weight ribbed knit,[Black]
7599,7599,shopbop,Slit Leather Pants,995.00,HLANG42154,0,"[\n 00, \n ...",None,[Black]


הריצה הראשונה נתקעה אחרי 7600 הורדות, נמשיך את הריצה הבאה מאותה הנקודה:

In [2]:
dfS=pd.read_csv("shopbop_link_test3.csv")

ratings=[]
prices=[]
product_names=[] 
brands=[] 
colors=[] 
sizes=[] 
fabrics=[] 
ids=[]

categorys=[] 

row = dict()
temp_list=[]

for num, s in enumerate(dfS["link S"]):
    if num>7600:
        
        driverS.get(s) 
        time.sleep(4)

        row = dict()
        row['num']=num
        soup = BeautifulSoup(driverS.page_source, 'html.parser')
        brands.append("shopbop")
        row['brand']= "shopbop"

        name, price, idp= name_price_id(soup)

        product_names.append(name)
        row['product_names']= name

        prices.append(price)
        row['prices']= price

        ids.append(idp)
        row['ids']= idp

        rating=ratingS(soup)
        ratings.append(rating)
        row['ratings']= rating

        size=sizeS(soup)
        sizes.append(size)
        row['sizes']= size

        fabric=fabricS(soup)
        fabrics.append(fabric)
        row['fabrics']= fabric

        color=colorS(soup)
        colors.append(color)
        row['colors']= color

        temp_list.append(row)

        if num%100==0:
            df = pd.DataFrame(temp_list)
            df.to_csv("shopbop_prodoct_info_temp3.csv", index=False)
        

נתקע שוב אחרי 4400 הורדות(12000) 

In [5]:
df

,num,brand,product_names,prices,ids,ratings,sizes,fabrics,colors
0,7601,shopbop,Hide Puffer,895.00,NANUS30098,5.0,"[\n XXS, \n ...","Fabric: Quilted, heavyweight faux leather",[Black]
1,7602,shopbop,Kacie 2 Piece Maxi Dress,375.00,BLACK30473,4.0,"[\n 0, \n ...",Fabric: Mid-weight stretch suiting,[Pacific Blue]
2,7603,shopbop,Brigitte Linen Mini Dress In Lime,260.00,SLEPR30070,0,"[\n XS, \n ...","Fabric: Lightweight, non-stretch linen weave",[Lime]
3,7604,shopbop,Oversize Crew Neck Tee,78.00,HRLOW40573,3.0,"[\n XS, \n ...",Fabric: Slubbed jersey,[White]
4,7605,shopbop,OVERSIZED SHIRT,310.00,SPRWM30128,0,"[\n XS, \n ...","Fabric: Lightweight, non-stretch silk poplin",[Striped Purple]
...,...,...,...,...,...,...,...,...,...
4395,11996,shopbop,Nic Bikini Top,12.37,MADEW45402,0,"[\n XXS, \n ...",Floral print,[Retro Floral]
4396,11997,shopbop,Isabel Balconette Bra,59.20,JNELL30061,0,"[\n 30C, \n ...",Sheer lace,[Emerald]
4397,11998,shopbop,Grenadins Bikini Top,37.50,FSCHH30060,0,"[\n XS, \n ...",Fish print,[Emerald Flying Fish]
4398,11999,shopbop,Crochet Tank,74.50,FRAME32046,0,"[\n XXS, \n ...",Fabric: Lightweight ribbed knit,[Kumquat]


In [3]:
dfS=pd.read_csv("shopbop_link_test3.csv")

ratings=[]
prices=[]
product_names=[] 
brands=[] 
colors=[] 
sizes=[] 
fabrics=[] 
ids=[]

categorys=[] 

row = dict()
temp_list=[]

for num, s in enumerate(dfS["link S"]):
    if num>12000:
        
        driverS.get(s) 
        time.sleep(4)

        row = dict()
        row['num']=num
        soup = BeautifulSoup(driverS.page_source, 'html.parser')
        brands.append("shopbop")
        row['brand']= "shopbop"

        name, price, idp= name_price_id(soup)

        product_names.append(name)
        row['product_names']= name

        prices.append(price)
        row['prices']= price

        ids.append(idp)
        row['ids']= idp

        rating=ratingS(soup)
        ratings.append(rating)
        row['ratings']= rating

        size=sizeS(soup)
        sizes.append(size)
        row['sizes']= size

        fabric=fabricS(soup)
        fabrics.append(fabric)
        row['fabrics']= fabric

        color=colorS(soup)
        colors.append(color)
        row['colors']= color

        temp_list.append(row)

        if num%100==0:
            df = pd.DataFrame(temp_list)
            df.to_csv("shopbop_prodoct_info_temp4.csv", index=False)
        

In [10]:
df

,num,brand,product_names,prices,ids,ratings,sizes,fabrics,colors
0,12001,shopbop,Maydum Skirt,161.25,LEKAS30241,0,"[\n XS, \n ...",None,[Charcoal]
1,12002,shopbop,Cassie Jeans,39.50,PISTO30162,0,"[\n 24, \n ...",None,[Snowfall Distressed]
2,12003,shopbop,Asher Backless Rib Dress,87.50,RAGBO46103,0,"[\n XXS, \n ...",Fabric: Mid-weight ribbed knit,[Black]
3,12004,shopbop,The Dazzler Hover Jeans,59.40,MOTHR21506,0,"[\n 23, \n ...",None,[Pale Peach]
4,12005,shopbop,Admiration Asymmetric Mini Dress,148.75,AJEEE30194,5.0,"[\n 4, \n ...",None,[Fuchsia]
...,...,...,...,...,...,...,...,...,...
795,12796,shopbop,Farrah Dress,64.50,YUMIK41051,0,"[\n XXS, \n ...",None,[Full Of Love Dusk]
796,12797,shopbop,The Seeker Skirt,19.80,BBDAK43536,0,"[\n XS, \n ...",Fabric: Lightweight lace,[White]
797,12798,shopbop,Midori Bikini Bottoms,31.50,VITAM30597,0,"[\n XS, \n ...",Ribbed knit,[Slate Shimmer Rib]
798,12799,shopbop,Willow Tank,12.00,SINCB30017,0,"[\n XS, \n ...",Fabric: Lightweight jersey,[Kiwi]


In [ ]:
נתקע עוד פעם אחרי שהוריד 800 (12800)

In [11]:
dfS=pd.read_csv("shopbop_link_test3.csv")

ratings=[]
prices=[]
product_names=[] 
brands=[] 
colors=[] 
sizes=[] 
fabrics=[] 
ids=[]

categorys=[] 

row = dict()
temp_list=[]

for num, s in enumerate(dfS["link S"]):
    if num>12800:
        
        driverS.get(s) 
        time.sleep(4)

        row = dict()
        row['num']=num
        soup = BeautifulSoup(driverS.page_source, 'html.parser')
        brands.append("shopbop")
        row['brand']= "shopbop"

        name, price, idp= name_price_id(soup)

        product_names.append(name)
        row['product_names']= name

        prices.append(price)
        row['prices']= price

        ids.append(idp)
        row['ids']= idp

        rating=ratingS(soup)
        ratings.append(rating)
        row['ratings']= rating

        size=sizeS(soup)
        sizes.append(size)
        row['sizes']= size

        fabric=fabricS(soup)
        fabrics.append(fabric)
        row['fabrics']= fabric

        color=colorS(soup)
        colors.append(color)
        row['colors']= color

        temp_list.append(row)

        if num%100==0:
            df = pd.DataFrame(temp_list)
            df.to_csv("shopbop_prodoct_info_temp5.csv", index=False)
        
df

,num,brand,product_names,prices,ids,ratings,sizes,fabrics,colors
0,12801,shopbop,Duchesses Satin Dress,229.50,NOTWO30724,0,"[\n 36, \n ...",None,[Pink]
1,12802,shopbop,Ava Sweatshirt,95.00,CHUFY30057,0,"[\n XS, \n ...",Fabric: Lightweight fleece,[Lacar Navy]
2,12803,shopbop,Recycled Seersucker Frill Lowrise Bikini Briefs,33.75,GANNI31556,0,"[\n 32, \n ...",Seersucker knit with plaid pattern,[Check Azure Blue]
3,12804,shopbop,Drawstring A Line Maxi Skirt,82.25,SWFFF30027,0,"[\n XS, \n ...","Fabric: Lightweight, non-stretch weave",[Tiger]
4,12805,shopbop,Ebony Panties,17.25,SUZBL30006,0,"[\n XS, \n ...",Jersey lined gusset,[White]
...,...,...,...,...,...,...,...,...,...
1295,14096,shopbop,Berdine Faux Leather Leggings,44.10,VELVE41623,0,"[\n XS, \n ...",Fabric: Mid-weight stretch faux leather,[Black]
1296,14097,shopbop,Malta Gown,347.50,CCONS30753,0,"[\n XXS, \n ...",None,[Black Check Plaid]
1297,14098,shopbop,Boiled Wool Mini Skirt,247.50,TIBDB30355,0,"[\n 0, \n ...",Fabric: Heavyweight non-stretch fleece,[Midnight Navy]
1298,14099,shopbop,Nathalie Dress,347.50,ULLAJ22017,0,"[\n P, \n ...",Fabric: Heavyweight knit,[Twilight]


In [12]:
dfS=pd.read_csv("shopbop_link_test3.csv")

ratings=[]
prices=[]
product_names=[] 
brands=[] 
colors=[] 
sizes=[] 
fabrics=[] 
ids=[]

categorys=[] 

row = dict()
temp_list=[]

for num, s in enumerate(dfS["link S"]):
    if num>14100:
        
        driverS.get(s) 
        time.sleep(4)

        row = dict()
        row['num']=num
        soup = BeautifulSoup(driverS.page_source, 'html.parser')
        brands.append("shopbop")
        row['brand']= "shopbop"

        name, price, idp= name_price_id(soup)

        product_names.append(name)
        row['product_names']= name

        prices.append(price)
        row['prices']= price

        ids.append(idp)
        row['ids']= idp

        rating=ratingS(soup)
        ratings.append(rating)
        row['ratings']= rating

        size=sizeS(soup)
        sizes.append(size)
        row['sizes']= size

        fabric=fabricS(soup)
        fabrics.append(fabric)
        row['fabrics']= fabric

        color=colorS(soup)
        colors.append(color)
        row['colors']= color

        temp_list.append(row)

        if num%1==0:
            df = pd.DataFrame(temp_list)
            df.to_csv("shopbop_prodoct_info_temp6.csv", index=False)
        
df

,num,brand,product_names,prices,ids,ratings,sizes,fabrics,colors
0,14101,shopbop,Orion One Piece Swimsuit,44.75,XIREN30663,0,"[\n S, \n ...",Scoop neck and tie shoulder straps,[Blue Sails]
1,14102,shopbop,Textural Hanelee Cardigan,191.25,THEOR44270,0,"[\n P, \n ...",Fabric: Mid-weight textured knit,[Heather Grey]
2,14103,shopbop,70s Bell Jeans,147.50,REDON30703,0,"[\n 24, \n ...",None,[Heritage Rinse]
3,14104,shopbop,Lucie Embroidered Cotton Midi Dress,59.20,MADEW45806,0,"[\n 00, \n ...",None,[Vintage Mulberry]
4,14105,shopbop,Odessa Wide Straight Jeans,140.00,MOUSS30201,0,"[\n 23, \n ...",None,[Black]
5,14106,shopbop,Lumire Bicolor Bandeau Set,131.00,OSERE30075,0,"[\n S, \n ...",Colorblock design with glitter finish,[Violet/Orange]


חיברנו את כל הטבלאות לטבלה אחת:

In [5]:
df1= pd.read_csv("shopbop_prodoct_info_temp2.csv")
df2= pd.read_csv("shopbop_prodoct_info_temp3.csv")

df3 = df1.append(df2, ignore_index=True)

for i in range(4,7):
    df1=pd.read_csv("shopbop_prodoct_info_temp"+str(i)+".csv")
    
    df4 = df3.append(df1, ignore_index=True)
    
    df3=df4

df4

C:\Users\linoy\AppData\Local\Temp\ipykernel_26168\54347521.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df1.append(df2, ignore_index=True)
C:\Users\linoy\AppData\Local\Temp\ipykernel_26168\54347521.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df4 = df3.append(df1, ignore_index=True)
C:\Users\linoy\AppData\Local\Temp\ipykernel_26168\54347521.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df4 = df3.append(df1, ignore_index=True)
C:\Users\linoy\AppData\Local\Temp\ipykernel_26168\54347521.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df4 = df3.append(df1, ignore_index=True)


,num,brand,product_names,prices,ids,ratings,sizes,fabrics,colors
0,0,shopbop,Lace Top,490.00,PACOO30471,0.0,"['\n 34', '\n ...","Fabric: Lightweight, stretch lace",['Black']
1,1,shopbop,Jupe Skirt,620.00,PACOO30468,0.0,"['\n 34', '\n ...","Fabric: Lightweight, stretch floral lace",['Black']
2,2,shopbop,Ella Dress,285.00,XIREN30738,0.0,"['\n XS', '\n ...",NaN,"['Electric Blue', 'Rococo Rose']"
3,3,shopbop,Rosalie Dress,840.00,LGVSG30040,0.0,"['\n 34', '\n ...",NaN,['Black']
4,4,shopbop,Wyatt Shirt,255.00,RAGBO46380,0.0,"['\n XXS', '\n ...","Fabric: Mid-weight, non-stretch denim",['Vintage Blue']
...,...,...,...,...,...,...,...,...,...
14102,14102,shopbop,Textural Hanelee Cardigan,191.25,THEOR44270,0.0,"['\n P', '\n ...",Fabric: Mid-weight textured knit,['Heather Grey']
14103,14103,shopbop,70s Bell Jeans,147.50,REDON30703,0.0,"['\n 24', '\n ...",NaN,['Heritage Rinse']
14104,14104,shopbop,Lucie Embroidered Cotton Midi Dress,59.20,MADEW45806,0.0,"['\n 00', '\n ...",NaN,['Vintage Mulberry']
14105,14105,shopbop,Odessa Wide Straight Jeans,140.00,MOUSS30201,0.0,"['\n 23', '\n ...",NaN,['Black']


In [6]:
df4.to_csv("shopbop_prodoct_info_final.csv", index=False)